# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weather = pd.read_csv("city_data.csv")
weather.head()


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,64.94,94,100,1.39,US,1662415686
1,1,avarua,-21.2078,-159.7750,77.05,78,100,17.27,CK,1662415687
2,2,tigil,57.8000,158.6667,45.23,69,0,4.16,RU,1662415687
3,3,severo-kurilsk,50.6789,156.1250,54.28,93,96,8.63,RU,1662415688
4,4,butaritari,3.0707,172.7902,81.52,79,76,16.02,KI,1662415688


In [7]:
# deleting the unnamed column
del weather["Unnamed: 0"]
weather.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.6001,-73.9662,64.94,94,100,1.39,US,1662415686
1,avarua,-21.2078,-159.7750,77.05,78,100,17.27,CK,1662415687
2,tigil,57.8000,158.6667,45.23,69,0,4.16,RU,1662415687
3,severo-kurilsk,50.6789,156.1250,54.28,93,96,8.63,RU,1662415688
4,butaritari,3.0707,172.7902,81.52,79,76,16.02,KI,1662415688


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
locations = weather[["Lat", "Lng"]]
# weight = weather["Humidity"]

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weather["Humidity"], 
                                 dissipating=False, max_intensity=10,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# checking for null values
weather = weather.dropna(how='any')
weather.count()

City          553
Lat           553
Lng           553
Max Temp      553
Humidity      553
Cloudiness    553
Wind Speed    553
Country       553
Date          553
dtype: int64

In [10]:
# setting ideal temperature range
clean_weather = weather.loc[weather["Max Temp"].between(70, 80)]
clean_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,avarua,-21.2078,-159.7750,77.05,78,100,17.27,CK,1662415687
7,ponta do sol,32.6667,-17.1000,72.23,73,99,2.44,PT,1662415690
10,nkhata bay,-11.6066,34.2907,70.83,51,0,5.39,MW,1662415693
11,itarema,-2.9248,-39.9167,78.55,81,35,17.58,BR,1662415693
13,hasaki,35.7333,140.8333,77.29,84,19,9.93,JP,1662415694
...,...,...,...,...,...,...,...,...,...
546,madimba,-10.4000,40.3333,70.90,83,9,7.36,TZ,1662415937
547,puerto baquerizo moreno,-0.9000,-89.6000,72.59,83,75,12.66,EC,1662415937
548,wanning,18.8003,110.3967,77.32,72,66,3.20,CN,1662415938
553,ende,-8.8432,121.6623,75.38,89,66,1.63,ID,1662415940


In [11]:
# setting ideal wind speed
clean_weather_2 = clean_weather.loc[clean_weather["Wind Speed"] < 10]
clean_weather_2.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,ponta do sol,32.6667,-17.1000,72.23,73,99,2.44,PT,1662415690
10,nkhata bay,-11.6066,34.2907,70.83,51,0,5.39,MW,1662415693
13,hasaki,35.7333,140.8333,77.29,84,19,9.93,JP,1662415694
19,puerto ayora,-0.7393,-90.3518,78.76,90,95,8.01,EC,1662415697
27,tiznit,29.5833,-9.5000,71.55,54,63,4.45,MA,1662415701


In [12]:
# setting no cloudiness
ideal = clean_weather_2.loc[clean_weather_2["Cloudiness"] == 0]
ideal.head()

# now the ideal parameters are set in new data frame called ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,nkhata bay,-11.6066,34.2907,70.83,51,0,5.39,MW,1662415693
51,korla,41.7597,86.1469,72.45,19,0,2.73,CN,1662415712
61,nantucket,41.2835,-70.0995,72.00,82,0,8.05,US,1662415682
143,hami,42.8000,93.4500,73.92,17,0,6.93,CN,1662415754
193,gat,31.6100,34.7642,77.68,78,0,2.37,IL,1662415776


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_df = []
ideal.insert(loc=9, column='Hotel Name', value=0)


In [14]:
ideal

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
10,nkhata bay,-11.6066,34.2907,70.83,51,0,5.39,MW,1662415693,0
51,korla,41.7597,86.1469,72.45,19,0,2.73,CN,1662415712,0
61,nantucket,41.2835,-70.0995,72.00,82,0,8.05,US,1662415682,0
143,hami,42.8000,93.4500,73.92,17,0,6.93,CN,1662415754,0
193,gat,31.6100,34.7642,77.68,78,0,2.37,IL,1662415776,0
232,sayat,38.7839,63.8803,78.76,14,0,7.81,TM,1662415793,0
283,nador,35.1740,-2.9287,77.25,57,0,1.14,MA,1662415822,0
317,perivolion,39.4206,20.0147,75.65,64,0,7.99,GR,1662415836,0
323,taft,31.7471,54.2088,72.59,15,0,5.01,IR,1662415839,0
329,angoche,-16.2325,39.9086,71.02,83,0,6.64,MZ,1662415841,0


In [15]:


params = {"key": g_key, "radius": 5000}
for index, row in ideal.iterrows():
    params["location"] = f"{Lat},{Lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    

    
# f converting intiger to string
    

NameError: name 'Lat' is not defined

In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
